In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
df = pd.read_csv('/content/drive/My Drive/Googlecolab/Fakenewsdata.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df = df.dropna()
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
df.reset_index(inplace=True)
df.head()

,index,id,title,author,text,label
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
X=df.drop(['label'],axis=1)
X.head()

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [7]:
Y = df['label']
Y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [8]:
len(X)

18285

In [9]:
len(Y)

18285

In [10]:
import nltk

In [11]:
from nltk.corpus import stopwords

In [12]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
from nltk.stem.porter import PorterStemmer 

In [14]:
import re

In [17]:
X['title'].dtype

dtype('O')

In [15]:
PS = PorterStemmer()
corpus = []
for i in range(0,len(X)):
  review = re.sub('[^a-zA-Z]'," ", X['title'][i])
  review = review.lower()
  review = review.split()

  review = [PS.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review) 

In [16]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [18]:
from sklearn.model_selection import train_test_split

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
CV = CountVectorizer(max_features = 5000, ngram_range = (1,3))
X = CV.fit_transform(corpus).toarray()

In [22]:
X.shape

(18285, 5000)

In [24]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = True)

In [25]:
CV.get_feature_names()[:20]

['abandon',
 'abc',
 'abc news',
 'abduct',
 'abe',
 'abedin',
 'abl',
 'abort',
 'abroad',
 'absolut',
 'abstain',
 'absurd',
 'abus',
 'abus new',
 'abus new york',
 'academi',
 'accept',
 'access',
 'access pipelin',
 'access pipelin protest']

In [26]:
CV.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.int64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 1.0,
 'max_features': 5000,
 'min_df': 1,
 'ngram_range': (1, 3),
 'preprocessor': None,
 'stop_words': None,
 'strip_accents': None,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': None,
 'vocabulary': None}

In [27]:
count_df = pd.DataFrame(X_train, columns = CV.get_feature_names())
count_df.head()

,abandon,abc,abc news,abduct,abe,abedin,abl,abort,abroad,absolut,abstain,absurd,abus,abus new,abus new york,academi,accept,access,access pipelin,access pipelin protest,accid,accident,accord,account,accus,accus trump,achiev,acknowledg,acknowledg emf,acknowledg emf damag,aclu,acquit,acquitt,acr,across,act,act like,act new,act new york,action,...,yahoo,yale,ye,year,year ago,year breitbart,year eve,year later,year new,year new york,year old,year old girl,year sinc,yemen,yet,yet anoth,yiannopoulo,yield,yo,york,york citi,york new,york new york,york state,york time,yorker,young,youth,youtub,zealand,zero,zika,zika viru,zionist,zone,zone new,zone new york,zoo,zu,zuckerberg
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [28]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()

In [29]:
from sklearn.metrics import accuracy_score

In [30]:
classifier.fit(X_train,Y_train)
pred = classifier.predict(X_test)
score = accuracy_score(Y_test, pred)

In [31]:
score

0.9020712510356255

In [32]:
from sklearn.metrics import confusion_matrix

In [33]:
confusion_matrix(Y_test, pred)

array([[3080,  361],
       [ 230, 2364]])

**Multinomial Classifier with Hyper parameter*

In [35]:
classifier1 = MultinomialNB(alpha = 0.1)

In [37]:
previous_score = 0
for alpha in np.arange(0,1,0.1):
  sub_classifier = MultinomialNB(alpha=alpha)
  sub_classifier.fit(X_train,Y_train)
  pred1 = sub_classifier.predict(X_test)
  score = accuracy_score(Y_test, pred1)
  if score>previous_score:
    classifier1 = sub_classifier
  print("Alpha: {}, Score : {}".format(alpha,score))


/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:507: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


Alpha: 0.0, Score : 0.8911350455675228
Alpha: 0.1, Score : 0.9030654515327258
Alpha: 0.2, Score : 0.9022369511184756
Alpha: 0.30000000000000004, Score : 0.903728251864126
Alpha: 0.4, Score : 0.903728251864126
Alpha: 0.5, Score : 0.9042253521126761
Alpha: 0.6000000000000001, Score : 0.903728251864126
Alpha: 0.7000000000000001, Score : 0.9024026512013256
Alpha: 0.8, Score : 0.9025683512841757
Alpha: 0.9, Score : 0.9030654515327258


In [39]:
feature_names = CV.get_feature_names()

In [40]:
classifier1.coef_[0]

array([ -9.09772001,  -8.7554338 ,  -9.32597866, ..., -10.79231573,
        -8.62009245,  -9.62224448])

In [41]:
sorted(zip(classifier.coef_[0], feature_names), reverse=True)[:20]

[(-4.008711700947771, 'trump'),
 (-4.33873710145436, 'hillari'),
 (-4.377542675876155, 'clinton'),
 (-4.86642849284322, 'elect'),
 (-5.18892263349876, 'new'),
 (-5.268965341172296, 'video'),
 (-5.277775970854451, 'comment'),
 (-5.341724695454725, 'us'),
 (-5.4460375420801075, 'hillari clinton'),
 (-5.4460375420801075, 'fbi'),
 (-5.488824817285316, 'war'),
 (-5.539255670912208, 'vote'),
 (-5.550816493313285, 'email'),
 (-5.586323181770194, 'obama'),
 (-5.604560769319975, 'world'),
 (-5.785656084390685, 'russia'),
 (-5.815509047540367, 'donald'),
 (-5.8462807062071205, 'day'),
 (-5.8462807062071205, 'america'),
 (-5.862029063175259, 'donald trump')]

In [42]:
sorted(zip(classifier.coef_[0], feature_names))[:5000]

[(-10.698310970126737, 'abroad'),
 (-10.698310970126737, 'abus new'),
 (-10.698310970126737, 'abus new york'),
 (-10.698310970126737, 'accid'),
 (-10.698310970126737, 'act new'),
 (-10.698310970126737, 'act new york'),
 (-10.698310970126737, 'advic'),
 (-10.698310970126737, 'advis new'),
 (-10.698310970126737, 'advis new york'),
 (-10.698310970126737, 'age new'),
 (-10.698310970126737, 'age new york'),
 (-10.698310970126737, 'agenda breitbart'),
 (-10.698310970126737, 'airlin'),
 (-10.698310970126737, 'aleppo new'),
 (-10.698310970126737, 'aleppo new york'),
 (-10.698310970126737, 'ali'),
 (-10.698310970126737, 'allegedli'),
 (-10.698310970126737, 'america breitbart'),
 (-10.698310970126737, 'america new'),
 (-10.698310970126737, 'america new york'),
 (-10.698310970126737, 'american breitbart'),
 (-10.698310970126737, 'american new'),
 (-10.698310970126737, 'american new york'),
 (-10.698310970126737, 'ami'),
 (-10.698310970126737, 'ami schumer'),
 (-10.698310970126737, 'amp'),
 (-10.6